### Embedding states |1>, |2>, |3>, |4>, & |5>

In [2]:
import numpy as np
from utils import DFS
states = DFS().getAllStates()

In [3]:

print('ensuring states are normalized (aka <s|s> ~ 1)')
for s in states:
    print(np.dot(s,s))

print('-----------------------')
total = 0
for i in range(len(states)-1):
    for j in range(len(states)-1):
        if i != j:
            temp = np.dot(states[i],states[j])
            total += temp

orthoStatement = 'First 4 states are orthogonal with each other' if total == 0 else 'First 4 states are - NOT - orthogonal with each other'
print(orthoStatement)

print('-----------------------')
i = len(states)-1
for j in range(len(states)):
    if i != j:
        temp = np.dot(states[i],states[j])
        print(f'<state5|state{j+1}> = {temp}')
        total += temp

print('Since dot product of states are close enough to zero, we conclude all states are orthogonal to each other')

ensuring states are normalized (aka <s|s> ~ 1)
1.0000000000000004
0.9999999999999999
0.9999999999999998
0.9999999999999998
1.0
-----------------------
First 4 states are orthogonal with each other
-----------------------
<state5|state1> = 0.0
<state5|state2> = 0.0
<state5|state3> = 0.0
<state5|state4> = 2.7755575615628914e-17
Since dot product of states are close enough to zero, we conclude all states are orthogonal to each other


### So above ^ I was able to prove that those states are correct. 
### --------------------------------------------------------
## Now below we are trying to use the fixed ansatz or circuit to see if we are encoding the circuit correctly
### Please edit the parts which say `TODO` and where you deem necessary

In [4]:
from scipy.linalg import expm
X = [[0,1],[1,0]]
Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
Z = [[1,0],[0,-1]]
I = np.eye(2)
Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))

U_ex = lambda p : expm(-1j*np.pi*p*H_ex)

amp_sqrd = lambda c : np.real(c*np.conjugate(c))

inputStates, expectedStates = DFS().getInitialTargetStates()
# evaluation function (minimization function)
def f_CNOT(U):
    total_c = 0
    for i in range(len(inputStates)):
        total_c += amp_sqrd(Udot(expectedStates[i],U,inputStates[i]))
    return np.sqrt(1-(1/4)*abs(total_c))

#### We need to do it in pennylane to make it easy to train ....
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

def square_loss(targets, predictions):
    loss = 0
    for t, p in zip(targets, predictions):
        loss += (t - p) ** 2
    loss = loss / len(targets)
    return 0.5*loss

dev = qml.device('default.qubit', wires=6)

p1 = np.arccos(-1/np.sqrt(3))/np.pi
p2 = np.arcsin(1/3)/np.pi


def fixedCNOTCircuit():
    # We know this CNOT works
    # Figure 1 from "Universal Quantum Computation and Leakage Reduction in the 3-Qubit Decoherence Free Subsystem"
    qml.QubitUnitary(U_ex(p1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(-1/2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[1,2])
    qml.QubitUnitary(U_ex(-1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[1,2])
    qml.QubitUnitary(U_ex(1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[1,2])
    qml.QubitUnitary(U_ex(-1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(-1/2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1-p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-p1), wires=[3,4])



@qml.qnode(dev)
def quantum_model(state_):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    fixedCNOTCircuit()
    return qml.state()

def target_function(U_prime):
    # approximated function
    return f_CNOT(U_prime)




for i, state in enumerate(states): ############################# TODO: I think this needs to be fixed because it's outputting the wrong shit <<<<<<<<<<<<<<<<<<<-------------------------
    newstate = quantum_model(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), expectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'for state{i+1}, we get |<expected_state|U_cnot|state{i+1}>|^2 = {np.real(dot_sqrd)}\t\t\t(should be = 1)')

op = qml.prod(fixedCNOTCircuit)() # finds the product for of these matrices
solved_U_cnot_matrix = qml.matrix(op)
print(f'If fixed matrix going into f_cnot = 0, then perfect! However, we get = {target_function(np.kron(I,solved_U_cnot_matrix))}')

for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.809134526419166			(should be = 1)
for state2, we get |<expected_state|U_cnot|state2>|^2 = 0.1596154735808357			(should be = 1)
for state3, we get |<expected_state|U_cnot|state3>|^2 = 0.015624999999999924			(should be = 1)
for state4, we get |<expected_state|U_cnot|state4>|^2 = 0.015625			(should be = 1)
for state5, we get |<expected_state|U_cnot|state5>|^2 = 0.1289648447247734			(should be = 1)
If fixed matrix going into f_cnot = 0, then perfect! However, we get = 0.7223783708629552


In [5]:
import random

def fill_array_with_random_values(size):
    random_values = [random.uniform(-1, 1) for _ in range(size)]
    return random_values
array_size = 18
param = fill_array_with_random_values(array_size)



def variableCNOTCircuit(param):
    # We know this CNOT works
    # Figure 1 from "Universal Quantum Computation and Leakage Reduction in the 3-Qubit Decoherence Free Subsystem"
    qml.QubitUnitary(U_ex(p1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[0]), wires=[2,3])
    qml.QubitUnitary(U_ex(p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[1]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[2]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[3]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[4]), wires=[1,2])
    qml.QubitUnitary(U_ex(param[5]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[6]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[7]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[8]), wires=[1,2])
    qml.QubitUnitary(U_ex(param[9]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[10]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[11]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[12]), wires=[1,2])
    qml.QubitUnitary(U_ex(param[13]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[14]), wires=[2,3])
    qml.QubitUnitary(U_ex(param[15]), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(param[16]), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(param[17]), wires=[2,3])
    qml.QubitUnitary(U_ex(1-p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-p1), wires=[3,4])



In [35]:
def func_target(param):
    solved_U_cnot_matrix = qml.matrix(variableCNOTCircuit)(param)
    return target_function(np.kron(I,solved_U_cnot_matrix))

In [40]:
import random

def fill_array_with_random_values(size):
    random_values = [random.uniform(-1, 1) for _ in range(size)]
    return random_values
array_size = 18
param = fill_array_with_random_values(array_size)

In [41]:
from scipy import optimize
bounds = optimize.Bounds(-1.0,1.0)
p_vals = optimize.minimize(func_target, param,method='L-BFGS-B',tol = 1e-5, bounds = bounds)

In [42]:
p_vals.x

array([-0.86394187,  1.        , -0.43826463, -0.299874  ,  0.08194724,
       -0.56093927, -0.6065316 , -1.        ,  1.        , -0.61456763,
       -0.91213267, -0.08056807,  0.5455069 ,  0.45206125,  0.3048947 ,
       -0.76530422, -0.43708586,  0.33315151])

In [43]:
optimised_params = p_vals.x

@qml.qnode(dev)
def quantum_model(state_):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    variableCNOTCircuit(optimised_params)
    return qml.state()

def target_function(U_prime):
    # approximated function
    return f_CNOT(U_prime)



for i, state in enumerate(states): ############################# TODO: I think this needs to be fixed because it's outputting the wrong shit <<<<<<<<<<<<<<<<<<<-------------------------
    newstate = quantum_model(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), expectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'for state{i+1}, we get |<expected_state|U_cnot|state{i+1}>|^2 = {np.real(dot_sqrd)}\t\t\t(should be = 1)')

op = qml.prod(variableCNOTCircuit)(optimised_params) # finds the product for of these matrices
solved_U_cnot_matrix = qml.matrix(op)
print(f'If fixed matrix going into f_cnot = 0, then perfect! However, we get = {target_function(np.kron(I,solved_U_cnot_matrix))}')

for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.051700861274214846			(should be = 1)
for state2, we get |<expected_state|U_cnot|state2>|^2 = 0.026899063527505963			(should be = 1)
for state3, we get |<expected_state|U_cnot|state3>|^2 = 0.04525200500795799			(should be = 1)
for state4, we get |<expected_state|U_cnot|state4>|^2 = 0.1306062420778541			(should be = 1)
for state5, we get |<expected_state|U_cnot|state5>|^2 = 0.08600562141106033			(should be = 1)
If fixed matrix going into f_cnot = 0, then perfect! However, we get = 0.3722738226470272


In [13]:
p_vals = [param]
cost = [func_target(param)] 
opt = qml.AdagradOptimizer()
max_iterations = 100 
conv_tol = 0.1 

for n in range(max_iterations):
    param, prev_cost = opt.step_and_cost(func_target, param) # need to fix the cost function not optimising for fnot
    cost.append(func_target(param))
    p_vals.append(param)

    conv = np.abs(cost[-1] - prev_cost)
    if n % 10 == 0:
        print(f"Step = {n},  Cost function = {cost[-1]:.8f} ")
    if conv >= conv_tol:
        break

print("\n" f"Final value of the cost function = {cost[-1]:.8f} ")

C:\Users\maxaa\anaconda3\Lib\site-packages\pennylane\_grad.py:112: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Step = 0,  Cost function = 0.83588464 
Step = 10,  Cost function = 0.83588464 
Step = 20,  Cost function = 0.83588464 
Step = 30,  Cost function = 0.83588464 
Step = 40,  Cost function = 0.83588464 
Step = 50,  Cost function = 0.83588464 
Step = 60,  Cost function = 0.83588464 
Step = 70,  Cost function = 0.83588464 
Step = 80,  Cost function = 0.83588464 
Step = 90,  Cost function = 0.83588464 

Final value of the cost function = 0.83588464 
